In [ ]:
pip install langchain langgraph langchain_community langchain_openai

In [1]:
import langchain
import langgraph
import langchain_community
import langchain_openai

In [ ]:
import langchain_community

help(langchain_community)

In [ ]:
pip install langchain_community

In [2]:
import langchain

#dir(langchain)
print(langchain.__version__)

0.3.27


In [3]:
import os, getpass

if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

In [3]:
import langchain.tools

langchain.__version__

'0.3.27'

In [5]:
import langchain.tools

#help(langchain.tools)

In [6]:
#Create a Tool (Example: get_weather)

from langchain.tools import tool

@tool   # decorator
def get_weather(city: str) -> str:
    """Returns a mock weather report for the given city."""
    return f"It's always sunny in {city} with 25°C!"

In [9]:
#help(tool)

In [ ]:
#Initialize GPT Model (via LangChain)

from langchain.chat_models import init_chat_model

model4 = init_chat_model("gpt-4.1", model_provider="openai")

In [4]:
#Initialize GPT Model (via LangChain)

from langchain.chat_models import init_chat_model

model5 = init_chat_model("gpt-5", model_provider="openai")

In [55]:
help(model5)

Help on ChatOpenAI in module langchain_openai.chat_models.base object:

class ChatOpenAI(BaseChatOpenAI)
 |  ChatOpenAI(*args: Any, name: Optional[str] = None, cache: Union[langchain_core.caches.BaseCache, bool, NoneType] = None, verbose: bool = <factory>, callbacks: Union[list[langchain_core.callbacks.base.BaseCallbackHandler], langchain_core.callbacks.base.BaseCallbackManager, NoneType] = None, tags: Optional[list[str]] = None, metadata: Optional[dict[str, Any]] = None, custom_get_token_ids: Optional[Callable[[str], list[int]]] = None, callback_manager: Optional[langchain_core.callbacks.base.BaseCallbackManager] = None, rate_limiter: Optional[langchain_core.rate_limiters.BaseRateLimiter] = None, disable_streaming: Union[bool, Literal['tool_calling']] = False, client: Any = None, async_client: Any = None, root_client: Any = None, root_async_client: Any = None, model: str = 'gpt-3.5-turbo', temperature: Optional[float] = None, model_kwargs: dict[str, typing.Any] = <factory>, api_key: O

In [10]:
#Create the ReAct Agent (with Tool Binding)
from langgraph.prebuilt import create_react_agent

tools = [get_weather]
agent_executor = create_react_agent(model=model5, tools=tools)

In [11]:
# Run the Agent Without Memory (Single Turn)

input_message = {"role": "user", "content": "What is the weather in Pune?"}
response = agent_executor.invoke({"messages": [input_message]})

for msg in response["messages"]:
    msg.pretty_print()

================================ Human Message =================================

What is the weather in Pune?
================================== Ai Message ==================================
Tool Calls:
  get_weather (call_g4p9Foujs0HuEXd7u1d7nJFx)
 Call ID: call_g4p9Foujs0HuEXd7u1d7nJFx
  Args:
    city: Pune
================================= Tool Message =================================
Name: get_weather

It's always sunny in Pune with 25°C!
================================== Ai Message ==================================

Here’s a quick update for Pune:
- Condition: Sunny
- Temperature: 25°C

Note: This is a mock report. For real-time weather, please check a reliable weather service or app.


In [58]:
#Add Memory for Multi-Turn Agent

from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()
agent_executor = create_react_agent(model=model5, tools=tools, checkpointer=memory)

config = {"configurable": {"thread_id": "weather-001"}}

# Turn 1
response1=agent_executor.invoke({"messages": [{"role": "user", "content": "My city is Nagpur"}]}, config)

for msg in response1["messages"]:
    msg.pretty_print()

================================ Human Message =================================

My city is Nagpur
================================== Ai Message ==================================

Great—thanks for sharing! How would you like me to help with Nagpur?

Options I can help with:
- Places to visit or a 1-day/2-day itinerary
- Best local food (Saoji spots, street food, cafés)
- Commute and metro info
- Housing and cost of living tips
- Nearby getaways (Pench, Ramtek) and seasonal advice

Tell me what you’re looking for, and I’ll tailor it to Nagpur.


In [59]:
# Turn 2
response2 = agent_executor.invoke({"messages": [{"role": "user", "content": "What's the weather where I live?"}]}, config)

for msg in response2["messages"]:
    msg.pretty_print()

================================ Human Message =================================

My city is Nagpur
================================== Ai Message ==================================

Great—thanks for sharing! How would you like me to help with Nagpur?

Options I can help with:
- Places to visit or a 1-day/2-day itinerary
- Best local food (Saoji spots, street food, cafés)
- Commute and metro info
- Housing and cost of living tips
- Nearby getaways (Pench, Ramtek) and seasonal advice

Tell me what you’re looking for, and I’ll tailor it to Nagpur.
================================ Human Message =================================

What's the weather where I live?
================================== Ai Message ==================================

I can’t access live weather data, but for Nagpur around this time (September) it’s monsoon: warm, very humid, with scattered showers or thunderstorms. Typical highs are about 31–33°C, lows 23–25°C, and rain is likely on several days, especially late af

What is Chain-of-Thought (CoT)?

•	Instead of jumping to the final answer, the model is instructed to think aloud by breaking the problem into logical reasoning steps.
•	This improves accuracy, interpretability, and often factual correctness, especially for complex or multi-step questions.

In [63]:
#Create the ReAct Agent (with Tool Binding)
from langgraph.prebuilt import create_react_agent

tools = [get_weather]
agent_executor = create_react_agent(model=model5, tools=tools)

Chain-of-Thought in Action:

The key here is the phrase:

"Think step-by-step about possible causes."

This triggers CoT reasoning, prompting the model to:
  • Break the problem into smaller steps.
  • Analyze logically.
  • Avoid jumping to conclusions.


In [ ]:
# Run the Agent Without Memory (Single Turn)
input_message = {"role": "user", "content": "Why Temperature has increased by 2 degree in a September Month in the Pune city. Think step-by-step about possible causes."}

response3 = agent_executor.invoke({"messages": [input_message]})
for msg in response3["messages"]:
    msg.pretty_print()

================================ Human Message =================================

Why Temperature has increased by 2 degree September Month in Pune city. Think step-by-step about possible causes.
================================== Ai Message ==================================

A 2°C bump in September in Pune is plausible and usually comes from a mix of monsoon variability and local factors. Here’s a step-by-step way to think about possible causes and how they interact:

1) Define the change
- What exactly rose by 2°C: daytime maximum, nighttime minimum, or daily mean?
- Compared to what baseline: last year, the 1981–2010/1991–2020 climatology, or the month-to-date average?

2) Cloud and rainfall anomaly
- Fewer clouds and a rainfall deficit increase sunshine and surface heating, lifting daytime maxima by 1–4°C.
- Conversely, more clouds and high humidity can keep nights warmer (higher minima) even if days aren’t unusually hot.

3) Monsoon phase and synoptic setup
- Break monsoon or wea

What is Plan-and-Execute (PnE)?

Plan-and-Execute is a powerful agent pattern that:
•	First plans a sequence of subtasks based on the user request.
•	Then executes each step, often using tools, APIs, or functions.
This improves reliability, transparency, and modularity in AI task solving.

Use Case:
Prompt: “Launch a weekend sale campaign across 3 top-selling categories.”
 
Step 1: Plan
The agent first thinks through all the required steps before doing anything:

PLAN:
To launch a weekend sale campaign across the top 3 categories, the following steps are required:

1. Identify the top 3 selling product categories for the last 30 days.
2. Retrieve current pricing and inventory levels for these categories.
3. Draft a weekend sale campaign with discounts and promo copy.
4. Submit the campaign for manager approval.

Step 2: Execute Each Step
Each planned step is then executed one by one, using appropriate tools or API functions:

✅ Step 1: Identify top categories
🛠️ Tool Call → get_top_categories(last_n_days=30, top_k=3)

Response:
["Groceries", "Electronics", "Personal Care"]

Step 2: Fetch pricing and stock
🛠️ Tool Call → get_pricing_and_inventory(["Groceries", "Electronics", "Personal Care"])
Response:
{
  "Groceries": {"avg_price": "$3.5", "in_stock": "High"},
  "Electronics": {"avg_price": "$120", "in_stock": "Moderate"},
  "Personal Care": {"avg_price": "$8", "in_stock": "High"}
}


✅ Step 3: Create campaign draft
🛠️ Tool Call → create_campaign_draft(...)
Response:
Drafted a weekend promo:  
🛒 Groceries – 15% off  
🔌 Electronics – 10% cashback  
🧼 Personal Care – Buy 1 Get 1 Free  

✅ Step 4: Submit for approval
🛠️ Tool Call → submit_campaign(draft_id="promo_2025_weekend_001")
Response:
✅ Campaign submitted for approval. Awaiting response from Marketing Lead.

✅ Final Response to User:
🎯 Campaign Planning & Execution Complete!

🔹 Categories Selected: Groceries, Electronics, Personal Care  
🔹 Campaign Draft: Discounts + Promo Copy Created  
🔹 Status: Submitted for approval (ID: promo_2025_weekend_001)

In [ ]:
#%%writefile weekend_sale_agent1.py

from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import MemorySaver
from langchain.chat_models import init_chat_model

# ✅ Define your tools with docstrings
@tool
def get_top_categories() -> list:
    """Returns top 3 selling product categories."""
    return ["Groceries", "Electronics", "Personal Care"]

@tool
def get_pricing_and_inventory() -> dict:
    """Returns pricing and inventory levels."""
    return {
        "Groceries": {"avg_price": "$3.5", "in_stock": "High"},
        "Electronics": {"avg_price": "$120", "in_stock": "Moderate"},
        "Personal Care": {"avg_price": "$8", "in_stock": "High"},
    }

@tool
def create_campaign_draft() -> str:
    """Creates a campaign draft with promotions."""
    return """
    Groceries – 15% off
    Electronics – 10% cashback
    Personal Care – Buy 1 Get 1 Free
    """

@tool
def submit_campaign() -> str:
    """Submits the campaign for approval."""
    return " Campaign submitted for approval. ID: promo_2025_weekend_001"

#  Load GPT-4 model (or GPT-3.5 for free tier)
#llm = ChatOpenAI(model="gpt-4", temperature=0.3)

model5 = init_chat_model("gpt-5", model_provider="openai")

# Register tools
tools = [get_top_categories, get_pricing_and_inventory, create_campaign_draft, submit_campaign]

# Create agent using LangGraph
memory = MemorySaver()
agent_executor = create_react_agent(model5, tools, checkpointer=memory)

# Set thread ID
config = {"configurable": {"thread_id": "campaign-001"}}

# Ask agent to plan and execute
input_msg = {"role": "user", "content": "Launch a weekend sale campaign across 3 top-selling categories."}

for step in agent_executor.stream({"messages": [input_msg]}, config, stream_mode="values"):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Launch a weekend sale campaign across 3 top-selling categories.
================================== Ai Message ==================================
Tool Calls:
  get_top_categories (call_0WPNBZhrwXeA9fGq73JSxzRB)
 Call ID: call_0WPNBZhrwXeA9fGq73JSxzRB
  Args:
  get_pricing_and_inventory (call_Ho28tkQuwwQtHVPMWWnOrn11)
 Call ID: call_Ho28tkQuwwQtHVPMWWnOrn11
  Args:
================================= Tool Message =================================
Name: get_pricing_and_inventory

{"Groceries": {"avg_price": "$3.5", "in_stock": "High"}, "Electronics": {"avg_price": "$120", "in_stock": "Moderate"}, "Personal Care": {"avg_price": "$8", "in_stock": "High"}}
================================== Ai Message ==================================
Tool Calls:
  create_campaign_draft (call_V69NhWqDSjJmFAFU8EeMkhtX)
 Call ID: call_V69NhWqDSjJmFAFU8EeMkhtX
  Args:
================================= Tool Message =================

In [66]:
# Ask agent to plan and execute
input_msg = {"role": "user", "content": "Launch a weekend sale campaign across 3 top-selling categories. Create Plan Report for Executives"}

for step in agent_executor.stream({"messages": [input_msg]}, config, stream_mode="values"):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Launch a weekend sale campaign across 3 top-selling categories. Create Plan Report for Executives
================================== Ai Message ==================================
Tool Calls:
  get_top_categories (call_LpqexRYX17QFrwIU7nz1s7Fq)
 Call ID: call_LpqexRYX17QFrwIU7nz1s7Fq
  Args:
  get_pricing_and_inventory (call_qReIfK12t331F19LIlk0iUqv)
 Call ID: call_qReIfK12t331F19LIlk0iUqv
  Args:
================================= Tool Message =================================
Name: get_pricing_and_inventory

{"Groceries": {"avg_price": "$3.5", "in_stock": "High"}, "Electronics": {"avg_price": "$120", "in_stock": "Moderate"}, "Personal Care": {"avg_price": "$8", "in_stock": "High"}}
================================== Ai Message ==================================
Tool Calls:
  create_campaign_draft (call_O5okjHsqyFgPH86f8AUpSOg1)
 Call ID: call_O5okjHsqyFgPH86f8AUpSOg1
  Args:
==============================

================================[1m Human Message [0m=================================

Launch a weekend sale campaign across 3 top-selling categories. Create Plan Report for Executives
==================================[1m Ai Message [0m==================================
Tool Calls:
  get_top_categories (call_LpqexRYX17QFrwIU7nz1s7Fq)
 Call ID: call_LpqexRYX17QFrwIU7nz1s7Fq
  Args:
  get_pricing_and_inventory (call_qReIfK12t331F19LIlk0iUqv)
 Call ID: call_qReIfK12t331F19LIlk0iUqv
  Args:
=================================[1m Tool Message [0m=================================
Name: get_pricing_and_inventory

{"Groceries": {"avg_price": "$3.5", "in_stock": "High"}, "Electronics": {"avg_price": "$120", "in_stock": "Moderate"}, "Personal Care": {"avg_price": "$8", "in_stock": "High"}}
==================================[1m Ai Message [0m==================================
Tool Calls:
  create_campaign_draft (call_O5okjHsqyFgPH86f8AUpSOg1)
 Call ID: call_O5okjHsqyFgPH86f8AUpSOg1
  Args:
=================================[1m Tool Message [0m=================================
Name: create_campaign_draft


    🛒 Groceries – 15% off
    🔌 Electronics – 10% cashback
    🧼 Personal Care – Buy 1 Get 1 Free
    
==================================[1m Ai Message [0m==================================
Tool Calls:
  submit_campaign (call_BnyPm82iZ0OQVFgzOdDZL1Nd)
 Call ID: call_BnyPm82iZ0OQVFgzOdDZL1Nd
  Args:
=================================[1m Tool Message [0m=================================
Name: submit_campaign

✅ Campaign submitted for approval. ID: promo_2025_weekend_001
==================================[1m Ai Message [0m==================================

Executive Plan Report — Weekend Sale Campaign (Top 3 Categories)

Summary
- Campaign ID: promo_2025_weekend_001 (submitted for approval)
- Scope: 3 top-selling categories — Groceries, Electronics, Personal Care
- Proposed window: Fri Sep 12, 12:01 AM to Sun Sep 14, 11:59 PM (local time)
- Promotions (current draft):
  - Groceries: 15% off
  - Electronics: 10% cashback
  - Personal Care: Buy 1 Get 1 Free
- Inventory readiness: Groceries High, Electronics Moderate, Personal Care High
- Average prices: Groceries $3.50, Electronics $120, Personal Care $8.00

Objectives
- Drive weekend traffic, unit velocity, and market share in top categories
- Lift total basket size and cross-category attachment
- Acquire/reactivate customers ahead of Q4

Operational Plan
- Targeting: All shoppers; prioritize loyalty members and lapsed 90-day customers via CRM
- Channels: Homepage hero, category landing pages, app banners, email, push, paid social/search
- Controls:
  - Per-customer limit: 5 redemptions per category
  - Cashback cap: $50 per order for Electronics
  - Exclusions: Doorbusters, preorders, gift cards, Apple/Sony MAP-protected SKUs, tobacco, alcohol where restricted
  - Stacking: No stacking with other sitewide offers; loyalty points accrue as normal
  - Returns: Cashback forfeited on returns; B1G1 returns require paired item or adjusted refund
- Legal/compliance: Promo copy with “while supplies last”, “select SKUs”, geographic restrictions; ADA-compliant creatives

Financial Snapshot (baseline vs projected, weekend only; directional, based on assumptions)
- Baseline weekend revenue (illustrative):
  - Groceries $1.00M, Electronics $0.60M, Personal Care $0.40M (Total $2.00M)
- Inventory note: Electronics stock moderate; ensure replenishment triggers and substitution rules for fast-movers
- Current draft projections (as submitted; assumes vendor funding at 40% for CPG, none for Electronics; halo from non-promoted categories included)
  - Uplift assumptions:
    - Groceries: +20% units, effective discount 15% (7.5% net after 50% vendor funding if secured)
    - Electronics: +12% units, 10% cashback (net 10%)
    - Personal Care: +30% units, effective discount 25% on participating units (net ~15% with 40% vendor funding)
  - Revenue impact on promoted items: approximately +2% to +5% combined
  - Gross profit impact on promoted items: approximately -20% to -40% without halo; net -10% to -15% after halo and vendor funding
  - Halo effects (traffic/basket lift on non-promoted): +4% to +10% revenue on remainder of catalog; GP +$30K to +$75K (assumes $3.0M non-promoted baseline at ~25% margin)
  - Net weekend outcome (range): Revenue +2% to +5%; Gross profit -3% to -12%
- Key sensitivities: Vendor funding secured %, Electronics sell-through at moderate inventory, B1G1 attach rate and mix

Recommendation to improve margin while preserving volume
- Groceries: Move from 15% off to 10% off with $25 basket minimum; secure 40%+ vendor funding
- Electronics: Reduce to 8% cashback, cap $50 per order; feature 50 SKUs with sufficient on-hand; add bundle offers (e.g., laptop + accessory 12% off)
- Personal Care: Change B1G1 to “Second at 50% off” on curated SKUs; require 2+ units
- Expected effect: Similar revenue lift; improves gross profit by ~300–600 bps vs current draft
- Decision: Approve “Optimized Variant” if margin guardrails are required; current draft can proceed if share/traffic is the primary objective

Marketing and Creative
- Messages:
  - Groceries: “Stock up for the weekend — 10–15% off essentials”
  - Electronics: “Limited-time cashback up to $50 on top tech”
  - Personal Care: “Buy more, save more: 50% off your second item”
- Deliverables: 3 hero banners, 12 category tiles, 6 email modules, 8 push variants
- A/B tests: Discount framing (percent vs dollars), urgency badges, order minimums, cashback cap messaging

Measurement and KPIs
- Primary: Incremental revenue, gross profit, contribution margin, campaign ROI
- Secondary: Traffic, conversion rate, AOV, units/order, category attach, new vs returning customer mix, redemption rate, cashback burn, returns
- Diagnostics: Inventory OOS hours, rainchecks/backorders, fraud/abuse rate, coupon leakage
- Attribution: Match-market holdout on 10% of traffic; media geo holdouts

Risks and Mitigations
- Stockouts (Electronics): Prioritize in-stock SKUs; dynamic de-promotion on low OOS thresholds; waitlist with delayed cashback
- Margin compression: Enforce caps/minimum baskets; vendor co-op funding; SKU-level guardrails
- Compliance/MAP: Pre-cleared SKU list; automated MAP monitor
- Fraud/abuse: Per-user caps, device/account velocity checks, cashback issuance after delivery window

Go-Live Checklist
- Confirm promo rules and caps in promo engine
- Verify SKU lists and inventory thresholds; enable back-up SKUs
- QA price tests on top 50 SKUs per category across web/app
- Load creative and schedule email/push; QA links and tracking
- Enable dashboards: real-time sales, GP, OOS, redemption
- Staff CS and social response; publish FAQ
- Final legal review of disclosures

Next Steps (owner • due)
- Finance: Validate funding commitments with top 10 CPG vendors • today
- Merchandising: Lock SKU lists and substitutes • today
- Marketing: Approve creatives; schedule email/push and paid • today
- Engineering: Implement promo rules and QA • today
- Analytics: Stand up dashboards and holdouts • today
- Ops: DC/store readiness, replenishment thresholds • today

Decision asks
- Approve promo structure: Current draft vs Optimized Variant
- Approve weekend window and caps (electronics cashback cap $50; per-customer 5 redemptions)
- Confirm vendor funding targets (≥40% for Groceries/Personal Care)

Status
- Campaign submitted for approval: promo_2025_weekend_001
- Ready to schedule and publish upon executive sign-off

If you’d like, I can:
- Switch to the Optimized Variant and resubmit
- Set the exact start/end times and push live upon approval
- Produce the creative set and the KPI dashboard links for launch day

In [7]:
# Run the Agent Without Memory (Single Turn)

input_message = {"role": "user", "content": "What is the weather in Pune?"}
response = agent_executor.invoke({"messages": [input_message]})

#print(response["messages"][0])

for msg in response["messages"]:
    msg.pretty_print()

================================ Human Message =================================

What is the weather in Pune?
================================== Ai Message ==================================
Tool Calls:
  get_weather (call_t3fgz3OPptrjov5yXnKfq3Nl)
 Call ID: call_t3fgz3OPptrjov5yXnKfq3Nl
  Args:
    city: Pune
================================= Tool Message =================================
Name: get_weather

It's always sunny in Pune with 25°C!
================================== Ai Message ==================================

The weather in Pune is sunny with a temperature of 25°C!


In [17]:
#Add Memory for Multi-Turn Agent

from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

agent_executor = create_react_agent(model=Amodel, tools=tools, checkpointer=memory)

config = {"configurable": {"thread_id": "weather-001"}}

# Turn 1
agent_executor.invoke({"messages": [{"role": "user", "content": "My city is Bangalore"}]}, config)

# Turn 2
response=agent_executor.invoke({"messages": [{"role": "user", "content": "What's the weather where I live?"}]}, config)

for msg in response["messages"]:
    msg.pretty_print()

================================ Human Message =================================

My city is Bangalore
================================== Ai Message ==================================

Bangalore, also known as Bengaluru, is the capital city of Karnataka, India. It's famous for its pleasant weather, tech industry, and vibrant culture. Would you like to know the current weather in Bangalore?
================================ Human Message =================================

What's the weather where I live?
================================== Ai Message ==================================
Tool Calls:
  get_weather (call_2PkBg10lh8xJvfF4foJ3c1gG)
 Call ID: call_2PkBg10lh8xJvfF4foJ3c1gG
  Args:
    city: Bangalore
================================= Tool Message =================================
Name: get_weather

It's always sunny in Bangalore with 25°C!
================================== Ai Message ==================================

The weather in Bangalore is sunny and pleasant with a tempera

In [3]:
#Create a Tool (Example: walmart_sales)

from langchain.tools import tool

@tool
def walmart_sales(city: str) -> str:
    """Returns a walmart sales report for the given city."""
    return f"Walmart Sales in {city} "

In [27]:
#Create the ReAct Agent (with Tool Binding)
from langgraph.prebuilt import create_react_agent

tools = [walmart_sales]
agent_sales_executor = create_react_agent(model=Amodel, tools=tools)

In [28]:
# Run the Agent Without Memory (Single Turn)

input_message = {"role": "user", "content": "How it is increased by 20% this month in chicago city."}
response = agent_sales_executor.invoke({"messages": [input_message]})

for msg in response["messages"]:
    msg.pretty_print()

================================ Human Message =================================

How it is increased by 20% this month in chicago city.
================================== Ai Message ==================================
Tool Calls:
  walmart_sales (call_98CbfIOSbsMITAIK5j3VCcps)
 Call ID: call_98CbfIOSbsMITAIK5j3VCcps
  Args:
    city: Chicago
================================= Tool Message =================================
Name: walmart_sales

Walmart Sales in Chicago 
================================== Ai Message ==================================

Could you clarify your question? Are you asking about a 20% increase in Walmart sales in Chicago this month, or something else?


In [29]:
#Use CoT with LangChain
query = {
    "role": "user",
    "content": "Walmart's sales increased by 20% this month. Think step-by-step about possible causes."
}
response = Amodel.invoke([query])
print(response.text())

Certainly! Let's think step-by-step about possible causes for Walmart's 20% sales increase this month:

---

### **1. Seasonal Factors**
- **Holidays or Events:** If this month coincided with major holidays (e.g., Thanksgiving, Christmas, or back-to-school season), customers may have increased spending on gifts, decorations, food, and supplies.
- **Weather Changes:** Seasonal shifts, such as colder weather, could drive demand for winter clothing, heating supplies, or holiday-related items.

---

### **2. Promotions and Discounts**
- **Sales Campaigns:** Walmart may have launched aggressive promotions, discounts, or "rollback" pricing on popular items, attracting more customers.
- **Special Events:** Events like "Black Friday," "Cyber Monday," or clearance sales could have boosted foot traffic and online sales.
- **Loyalty Programs:** Enhanced loyalty rewards or exclusive member discounts (e.g., Walmart+ perks) might have incentivized repeat purchases.

---

### **3. Economic Factors**


In [30]:
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import MemorySaver

Use Case:
Prompt: “Launch a weekend sale campaign across 3 top-selling categories.”

Step 1: Plan
The agent first thinks through all the required steps before doing anything:

PLAN:
To launch a weekend sale campaign across the top 3 categories, the following steps are required:

1. Identify the top 3 selling product categories for the last 30 days.
2. Retrieve current pricing and inventory levels for these categories.
3. Draft a weekend sale campaign with discounts and promo copy.
4. Submit the campaign for manager approval.

In [ ]:
%%writefile weekend_sale_agent1.py

from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import MemorySaver

#  Define your tools with docstrings
@tool
def get_top_categories() -> list:
    """Returns top 3 selling product categories."""
    return ["Groceries", "Electronics", "Personal Care"]

@tool
def get_pricing_and_inventory() -> dict:
    """Returns pricing and inventory levels."""
    return {
        "Groceries": {"avg_price": "$3.5", "in_stock": "High"},
        "Electronics": {"avg_price": "$120", "in_stock": "Moderate"},
        "Personal Care": {"avg_price": "$8", "in_stock": "High"},
    }

@tool
def create_campaign_draft() -> str:
    """Creates a campaign draft with promotions."""
    return """
    🛒 Groceries – 15% off
    🔌 Electronics – 10% cashback
    🧼 Personal Care – Buy 1 Get 1 Free
    """

@tool
def submit_campaign() -> str:
    """Submits the campaign for approval."""
    return "✅ Campaign submitted for approval. ID: promo_2025_weekend_001"

# ✅ Load GPT-4 model (or GPT-3.5 for free tier)
#llm = ChatOpenAI(model="gpt-4", temperature=0.3)

# Register tools
tools = [get_top_categories, get_pricing_and_inventory, create_campaign_draft, submit_campaign]

# Create agent using LangGraph
memory = MemorySaver()
agent_executor = create_react_agent(Amodel, tools, checkpointer=memory)

# Set thread ID
config = {"configurable": {"thread_id": "campaign-001"}}

# Ask agent to plan and execute
input_msg = {"role": "user", "content": "Launch a weekend sale campaign across 3 top-selling categories."}

for step in agent_executor.stream({"messages": [input_msg]}, config, stream_mode="values"):
    step["messages"][-1].pretty_print()

Overwriting weekend_sale_agent1.py
